In [16]:
import streamlit as st
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [17]:
search_for = "gents saloon"
search_for = search_for.replace(" ", "+")

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", 
            "Accept-Encoding":"gzip, deflate",
            "Accept-Language":"en,en-IN;q=0.9,en-US;q=0.8,hi;q=0.7",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
            "DNT":"1",
            "Connection":"close", 
            "Upgrade-Insecure-Requests":"1"}

In [18]:
latitude, longitude = 23.358402061376122, 85.30798006641939

# zoom = 16
zoom = 16.01
# height = 4700

delta_lat = 0.027
delta_long = delta_lat/np.cos(delta_lat*np.pi/180)

latlonglist = [(latitude+i, longitude+j) for i in [k*delta_lat for k in range(-2, 3)] 
                for j in [k*delta_long for k in range(-2, 3)]]

queryData = []

In [19]:
loopcounter = 0
for (lat, long) in latlonglist:
        
        link = ("https://www.google.co.in/maps/search/" + search_for + "/@" + str(round(lat, 7)) + "," + 
                str(round(long, 7)) + "," + str(zoom)+ "z/data=!4m4!2m3!5m1!4e3!6e5?hl=en&authuser=0")

        html_text = requests.get(link, headers=headers).text

        startList = [m.start() for m in re.finditer(r"\d+ reviews", html_text)]
        strchunks = [html_text[startList[i]:startList[i+1]] for i in range(0, len(startList)-1)]
        strchunks.append(html_text[startList[-1]:])

        for i in strchunks:
                chunckedlist = i.split(",")
                raters = int(re.findall("\d+", chunckedlist[7])[0])
                if raters < 30:
                        continue
                rating = float(chunckedlist[6])
                if rating < 4.0:
                        continue

                name = ""
                prevname = ""
                namelist = []
                counter = 0
                found = 0
                for i in chunckedlist[:100]:
                        if (i.startswith('\\"0x')) and (found==0):
                                found = 1
                                partCounter = counter
                        if (name=="") and (i.startswith('\\"') and (not (i.endswith('\\"') or i.endswith('\\"]')))):
                                name = i
                        elif (name != "") and (not i.endswith('\\"')):
                                name = name + i
                        elif (name != "") and (i.endswith('\\"')):
                                name = name + i
                                if len(name) >= len(prevname):
                                        prevname = name
                                        name = ""
                        counter += 1
                descrs = prevname.replace("\\", '').replace('"', '')
                loclat = float(re.findall("-?\d+\.?\d*", chunckedlist[partCounter-2])[0])
                loclong = float(re.findall("-?\d+\.?\d*", chunckedlist[partCounter-1])[0])
                if ((abs(abs(loclat)-abs(latitude)) > 0.1) or (abs(abs(loclong)-abs(longitude)) > 0.1)):
                        continue
                queryData.append([descrs, rating, raters, loclat, loclong])
        loopcounter += 1
        print(f"Completed {loopcounter} out of {len(latlonglist)}")
        print(link)

df = pd.DataFrame(queryData, columns=['Descrs', 'Rating', 'Raters', 'Latitude', 'Longitude'])
df = df.drop_duplicates()

df['Scaled Rating'] = df['Rating']*(1 - np.power(1.25, -1*np.sqrt(df['Raters'])))
df = df.sort_values("Scaled Rating", ascending=False)

df["Dist"] = (np.sqrt((((df['Latitude']-latitude)*111.3188)**2) + 
                      (((df['Longitude']-longitude)*np.cos(latitude*0.0174)*111.3188)**2)))

df["Scaled Dist Rating"] = df["Scaled Rating"]*(1 - np.power(1.25, -11.1/df["Dist"]))

df.to_csv("result.csv", index=False)
midLat = (df['Latitude']*df['Scaled Rating']).mean()/(df['Scaled Rating'].mean())
midLong = (df['Longitude']*df['Scaled Rating']).mean()/(df['Scaled Rating'].mean())
print(midLat, midLong)

Completed 1 out of 25
https://www.google.co.in/maps/search/gents+saloon/@23.3044021,85.2539801,16.01z/data=!4m4!2m3!5m1!4e3!6e5?hl=en&authuser=0
Completed 2 out of 25
https://www.google.co.in/maps/search/gents+saloon/@23.3044021,85.2809801,16.01z/data=!4m4!2m3!5m1!4e3!6e5?hl=en&authuser=0
Completed 3 out of 25
https://www.google.co.in/maps/search/gents+saloon/@23.3044021,85.3079801,16.01z/data=!4m4!2m3!5m1!4e3!6e5?hl=en&authuser=0
Completed 4 out of 25
https://www.google.co.in/maps/search/gents+saloon/@23.3044021,85.3349801,16.01z/data=!4m4!2m3!5m1!4e3!6e5?hl=en&authuser=0
Completed 5 out of 25
https://www.google.co.in/maps/search/gents+saloon/@23.3044021,85.3619801,16.01z/data=!4m4!2m3!5m1!4e3!6e5?hl=en&authuser=0
Completed 6 out of 25
https://www.google.co.in/maps/search/gents+saloon/@23.3314021,85.2539801,16.01z/data=!4m4!2m3!5m1!4e3!6e5?hl=en&authuser=0
Completed 7 out of 25
https://www.google.co.in/maps/search/gents+saloon/@23.3314021,85.2809801,16.01z/data=!4m4!2m3!5m1!4e3!6e5?hl

In [ ]:
## Bangalore

# len(df) @ 0.012 = 123
# len(df) @ 0.018 = 182
# len(df) @ 0.024 = 222
# len(df) @ 0.027 = 233
# len(df) @ 0.030 = 236
# len(df) @ 0.034 = 237
# len(df) @ 0.035 = 242
# len(df) @ 0.036 = 238
# len(df) @ 0.040 = 234

In [ ]:
## Indore

# len(df) @ 0.015 = 112
# len(df) @ 0.020 = 114
# len(df) @ 0.023 = 114
# len(df) @ 0.025 = 117
# len(df) @ 0.027 = 118
# len(df) @ 0.028 = 116
# len(df) @ 0.030 = 111
# len(df) @ 0.035 = 102
# len(df) @ 0.040 = 87

In [ ]:
## Jaipur

# len(df) @ 0.020 = 174
# len(df) @ 0.027 = 174
# len(df) @ 0.035 = 162

# ZOOM TEST

# 15.01 = 150
# 15.50 = 171
# 16.01 = 180
# 16.25 = 175
# 16.50 = 178
# 17.01 = 174
# 17.50 = 172
# 18.01 = 169

In [ ]:
## Bhubneshwar

# 16.01 = 68
# 17.01 = 66

In [ ]:
link

'https://www.google.co.in/maps/search/School/@20.329009,85.888133,16.01z/data=!4m4!2m3!5m1!4e3!6e5?hl=en&authuser=0'

In [ ]:
len(df)

68

In [ ]:
"ASTHA School Of Management Plot no- 261 Panchamukhi Vihar Phase-II Atala Balianta Bhubaneswar Odisha 752101".replace(" ", "+")

'ASTHA+School+Of+Management+Plot+no-+261+Panchamukhi+Vihar+Phase-II+Atala+Balianta+Bhubaneswar+Odisha+752101'

In [ ]:
df = pd.read_csv("/Users/snande/Personal/Projects/mapCrawler/result.csv")

In [ ]:
link = "https://www.google.co.in/search?q=google+maps+" + df.iloc[5, 0].replace(" ", "+") + "&tbm=isch"
html_text = requests.get(link, headers=headers).text
re.findall("http[^,]*jpg", html_text)[:15]

['https://www.yayskool.com/images/school/sainik-school-khurda-19606316.jpg',
 'http://photos.wikimapia.org/p/00/02/05/33/63_big.jpg',
 'https://www.yayskool.com/images/school/stxaviers-high-school-khurda-14206331.jpg',
 'https://sainikschoolbhubaneswar.org/wp-content/uploads/2022/04/COLONEL-BALU-BHARATH.jpg',
 'https://www.yayskool.com/images/school/st-xaviers-high-school-khurda-55256336.jpg',
 'https://sainikschoolbhubaneswar.org/wp-content/uploads/2022/10/AISSEE-2023.jpg',
 'https://www.yayskool.com/images/school/times-scholars-gurukul-khurda-32116338.jpg',
 'https://www.yayskool.com/images/school/venkateswar-english-medium-school-khurda-71086340.jpg',
 'https://sainikschoolbhubaneswar.org/wp-content/uploads/2022/12/pariksha-pe-charcha.jpg',
 'https://www.360hoardings.com/images_backup/Torrentadv-121.jpg',
 'https://sainikschoolbhubaneswar.org/wp-content/uploads/2021/08/Capt-Ashutosh-300x206.jpg',
 'https://www.360hoardings.com/images_backup/Cbb-03.jpg',
 'http://101bhubaneswar.com/w

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", 
            "Accept-Encoding":"gzip, deflate",
            "Accept-Language":"en,en-IN;q=0.9,en-US;q=0.8,hi;q=0.7",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
            "DNT":"1",
            "Connection":"close", 
            "Upgrade-Insecure-Requests":"1"}
            
html_text = requests.get(link, headers=headers).text

In [ ]:
soup = BeautifulSoup(html_text, "html.parser")

In [ ]:
import re

In [ ]:
link

"https://www.google.co.in/search?q=google+maps+7RHV+938+St.Joseph's+High+School+Bhubaneswar+Madhusudan+Marg+kharvel+Nagar+Bhubaneswar+Odisha&tbm=isch"

['https://smarteduguide.com/wp-content/uploads/2019/11/St-Josephs-High-School-Bhubaneswar.jpg',
 'https://akam.cdn.jdmagicbox.com/images/detail/bg5.jpg']

In [ ]:
soup.find("a", class_="wXeWr islib nfEiy")

<a class="wXeWr islib nfEiy" data-nav="1" jsaction="J9iaEb;mousedown:npT2md; touchstart:npT2md;" jsname="sTFXNd" role="button" style="height:180px;" tabindex="0"><div class="bRMDJf islir" jsname="DeysSe" style=""></div><div class="c7cjWc mvjhOe"></div></a>

In [ ]:
[i for i in soup.findAll("a", class_="wXeWr islib nfEiy") if i["href"]]

KeyError: 'href'

In [ ]:
link = "https://www.google.com/search?q="+df.iloc[9, 0].replace(" ", "+")
html_text = requests.get(link, headers=headers).text
soup = BeautifulSoup(html_text, "html.parser")

In [ ]:
soup.find("g-img", class_="ZGomKf").find("img")["src"]

'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='

In [20]:
# importing geopy library
from geopy.geocoders import Nominatim

# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")

# entering the location name
getLoc = loc.geocode("Dinbandhu tower, Ranchi")

# printing address
print(getLoc.address)

# printing latitude and longitude
print("Latitude = ", getLoc.latitude, "\n")
print("Longitude = ", getLoc.longitude)

AttributeError: 'NoneType' object has no attribute 'address'

In [37]:
df.sample(1).iloc[:, [3,4]].iloc[0].to_list()

[23.3698963, 85.3336882]

In [45]:
df.sample(1).iloc[0,0]

'Life Style Next - Men’s Salon H.B.Road Udhav Babu Lane Tharpakhna New Barhi Toli Ranchi Jharkhand 834001'

In [43]:
loc.reverse("27.1750123, 78.04209683661315")

Location(Taj Mahal, Taj Mahal Internal Path, Taj Ganj, Agra, Agra division, Uttar Pradesh, 282006, India, (27.1750123, 78.04209683661315, 0.0))

In [64]:
import pandas as pd
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")

import plotly.express as px

fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="City", hover_data=["State", "Population"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

URLError: <urlopen error [Errno 60] Operation timed out>

In [69]:
import plotly.express as px
fig = px.scatter_mapbox(df, lat="lat", lon="lng")
fig.show()

In [26]:
loc.geocode("Dinbandhu apartments ITI Ranchi")

In [52]:
loc.geocode(df.sample(1).iloc[0,0])

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("/Users/snande/Personal/Projects/mapCrawler/worldcities.csv")

In [14]:
sorted(df["country"].unique())[93]

'India'

In [15]:
statelist = sorted(df.loc[df["country"]=="India", "admin_name"].unique())

In [18]:
statelist[6]

'Chhattīsgarh'

In [26]:
lat, lng = df.loc[(df["country"]=="India") & (df["admin_name"]=="Chhattīsgarh")& (df["city_ascii"]=="Raipur"), ["lat", "lng"]].iloc[0]

In [27]:
lat

'lat'

In [28]:
lng

'lng'

In [62]:
df.loc[df["country"]=="Pakistan", "admin_name"].unique()

array(['Sindh', 'Punjab', 'Khyber Pakhtunkhwa', 'Islāmābād',
       'Balochistān', 'Azad Kashmir', 'Gilgit-Baltistan'], dtype=object)

In [ ]:
df[]

In [63]:
df

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6839,139.7744,Japan,JP,JPN,Tōkyō,primary,39105000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,35362000.0,1360771077
2,Delhi,Delhi,28.6667,77.2167,India,IN,IND,Delhi,admin,31870000.0,1356872604
3,Manila,Manila,14.6000,120.9833,Philippines,PH,PHL,Manila,primary,23971000.0,1608618140
4,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22495000.0,1076532519
...,...,...,...,...,...,...,...,...,...,...,...
42900,Tukchi,Tukchi,57.3670,139.5000,Russia,RU,RUS,Khabarovskiy Kray,NaN,10.0,1643472801
42901,Numto,Numto,63.6667,71.3333,Russia,RU,RUS,Khanty-Mansiyskiy Avtonomnyy Okrug-Yugra,NaN,10.0,1643985006
42902,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1304217709
42903,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491


In [1]:
import folium
import streamlit as st

from streamlit_folium import st_folium

# center on Liberty Bell, add marker
m = folium.Map(location=[39.949610, -75.150282], zoom_start=16)
# folium.Marker(
#     [39.949610, -75.150282], popup="Liberty Bell", tooltip="Liberty Bell"
# ).add_to(m)

folium.CircleMarker().add_to(m)

# call to render Folium map in Streamlit
st_data = st_folium(m, width=725)

2023-01-09 21:19:05.503 
  command:

    streamlit run /Users/snande/opt/anaconda3/envs/personal-mapCrawler/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [2]:
x = {'lat': 17.58748778209402, 'lng': 80.25337010622026}
x.values()

dict_values([17.58748778209402, 80.25337010622026])

In [33]:
import folium

m = folium.Map(location=[22.9524142, 81.9137491], zoom_start=5)

# tile1 = folium.TileLayer(
#         tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
#         attr = 'Google',
#         name = 'Google Satellite',
#         overlay = True,
#         control = True
#     ).add_to(m)

m2 = folium.Map(location=[22.9524142, 81.9137491], zoom_start=5, tiles=None, 
                zoom_control=False, dragging=False, scrollWheelZoom=False)

c = folium.vector_layers.CircleMarker(location=[22.9524142, 81.9137491], radius=200).add_to(m)

m2.add_to(m)

m.

In [2]:
{'last_clicked': None, 'last_object_clicked': None, 'all_drawings': None, 'last_active_drawing': None, 'bounds': {'_southWest': {'lat': 0.9027915134100478, 'lng': 63.592865020036704}, '_northEast': {'lat': 30.165812248990626, 'lng': 85.5655212700367}}, 'zoom': 5, 'last_circle_radius': None, 'last_circle_polygon': None, 'center': {'lat': 16.068802747075274, 'lng': 74.57919314503671}}["center"]

{'lat': 16.068802747075274, 'lng': 74.57919314503671}

In [4]:
from streamlit_folium import st_folium


In [5]:
help(st_folium)

Help on function st_folium in module streamlit_folium:

st_folium(fig: 'folium.MacroElement', key: 'str | None' = None, height: 'int' = 700, width: 'int' = 500, returned_objects: 'Iterable[str] | None' = None, zoom: 'int | None' = None, center: 'tuple[float, float] | None' = None, feature_group_to_add: 'folium.FeatureGroup | None' = None)
    Display a Folium object in Streamlit, returning data as user interacts
    with app.
    Parameters
    ----------
    fig  : folium.Map or folium.Figure
        Geospatial visualization to render
    key: str or None
        An optional key that uniquely identifies this component. If this is
        None, and the component's arguments are changed, the component will
        be re-mounted in the Streamlit frontend and lose its current state.
    returned_objects: Iterable
        A list of folium objects (as keys of the returned dictionart) that will be
        returned to the user when they interact with the map. If None, all folium
        objec

In [24]:
import folium

m = folium.Map(location=[22.9524142, 81.9137491], zoom_start=5, tiles=None)

tile1 = folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ).add_to(m)


tile2 = (folium.TileLayer(
            # location=[0, 0], 
            attr = 'Custom',
            name = 'Google Satellite',
            tiles=None, 
            # no_touch=True,
            # disable_3d=True, 
            # zoom_control=False, 
            # max_bounds=True, 
            # dragging=False, 
            # scrollWheelZoom=False
            overlay = True,
            )
            .add_to(m))

# x = folium.map.CustomPane(name="static")

c = folium.vector_layers.CircleMarker(location=[0,0], radius=200)
c.add_to(tile2)
# l = (folium.raster_layers
#     .ImageOverlay("/Users/snande/Personal/Projects/mapCrawler/Circle_(transparent).png", ))
# x.add_to(m)


m

AttributeError: 'NoneType' object has no attribute 'lower'